<a href="https://colab.research.google.com/github/Adonyo-Emmanuel/Novel-Data-Visualisation-for-GWAS-Datasets/blob/master/3D_Manhattan_Plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This notebook parses GWAS summary statistics and generates 3D Manhattan plots
#### `holoviews` version

In [3]:
import pandas as pd
import numpy as np
import os
#from holoviews import pandas as hpd
#import holoviews as hv
#from holoviews import opts
#import interfacePandas as ip
#hv.extension('bokeh', 'matplotlib', 'plotly')
#hv.output(backend='matplotlib', holomap='widgets')
#%reload_ext holoviews.ipython
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from bioservices.kegg import KEGG
import pickle

ModuleNotFoundError: ignored

In [ ]:
pwd() 

'C:\\Users\\User\\3D_Manhattan_plot'

### Parse data and set up axes properties:

In [ ]:
GWADataPath = 'Data' #variable to the datapath directory
TmpPath = 'Tmp' # create the TemPath variable 

In [ ]:
FName = 'TraitAttributesDF.pckl' # Name of the data fiiles 
TraitAttributesDF = None
if(os.path.exists(os.path.join(GWADataPath, FName))):
    TraitAttributesDF = pd.read_pickle(
         os.path.join(GWADataPath,
                      FName))
    del FName
else:
    FName = 'ENGAGEBioCratesMetaboliteClasses.tsv'
    TraitAttributesDF = pandas.read_csv(os.path.join(GWADataPath,
                                                     FName),
                                        index_col=None,
                                        sep='\t')
    TraitAttributesDF.rename(columns={'# P': 'Index'},
                             inplace=True)
    del FName

    TraitFNameDF = pd.DataFrame(data=os.listdir(GWADataPath),
                                    columns=['FName'])
    TraitFNameDF.insert(loc=0,
                        column='Path',
                        value=GWADataPath)
    Filter = TraitFNameDF['FName'].str.endswith('.csv')
    TraitFNameDF = TraitFNameDF[Filter].copy()
    del Filter
    TraitFNameDF.reset_index(drop=True,
                             inplace=True)
    TraitFNameDF.insert(loc=0,
                        column='Trait',
                        value=TraitFNameDF['FName'].str.split('_',
                                                              expand=True)[2])
    TraitAttributesDF = TraitAttributesDF.merge(
        right=TraitFNameDF,
        left_on='ENGAGEName',
        right_on='Trait',
        how='right')
    del TraitFNameDF
    TraitAttributesDF['Index'] = TraitAttributesDF.index.values + 1

    FName = 'TraitAttributesDF.pckl'
    TraitAttributesDF.to_pickle(os.path.join(TmpPath,
                                             FName))
    TraitAttributesDF.to_pickle(os.path.join(GWADataPath,
                                             FName))
    del FName

In [ ]:
FName = 'GWASummaryStatisticsDF.pckl'
GWASummaryStatisticsDF = None
AxesDF = None
if(os.path.exists(os.path.join(GWADataPath,
                               FName))):
    GWASummaryStatisticsDF = pd.read_pickle(
         os.path.join(TmpPath,
                      FName))
    del FName
    FName = 'AxesDF.pckl'
    AxesDF = pd.read_pickle(
         os.path.join(GWADataPath,
                      FName))
    del FName
else:
    GWASummaryStatisticsDF = pd.DataFrame(columns=['Trait',
                                                       'MarkerName',
                                                       'Allele1',
                                                       'Allele2',
                                                       'Freq1',
                                                       'FreqSE',
                                                       'MinFreq',
                                                       'MaxFreq',
                                                       'Effect',
                                                       'StdErr',
                                                       'P-value',
                                                       'Direction',
                                                       'HetISq',
                                                       'HetChiSq',
                                                       'HetDf',
                                                       'HetPVal'])
    for i in TraitAttributesDF.index:
        Trait = TraitAttributesDF.loc[i,
                                      'Trait']
        TmpDF = pd.read_csv(
            os.path.join(TraitAttributesDF.loc[i,
                                               'Path'],
                         TraitAttributesDF.loc[i,
                                               'FName']),
            index_col=None,
            sep=',')
        TmpDF.insert(loc=0,
                     column='Trait',
                     value=Trait)
        GWASummaryStatisticsDF = GWASummaryStatisticsDF.append(TmpDF) 
        del TmpDF
    GWASummaryStatisticsDF['Allele1'] = GWASummaryStatisticsDF['Allele1'].str.upper()
    GWASummaryStatisticsDF['Allele2'] = GWASummaryStatisticsDF['Allele2'].str.upper()
    GWASummaryStatisticsDF.insert(loc=0,
                                  column='Index',
                                  value=GWASummaryStatisticsDF.index.values)
    
    FName = 'SNPInfo_MergedCorrected.txt.gz'
    VariantInfoDF = pd.read_csv(os.path.join(GWADataPath,
                                                 FName),
                                    index_col=None,
                                    sep=' ')
    del FName
    
    GWASummaryStatisticsDF = GWASummaryStatisticsDF.merge(
        right=VariantInfoDF,
        left_on='MarkerName',
        right_on='SNPID',
        how='left')
    
    GWASummaryStatisticsDF.insert(loc=5,
                                  column='Pos',
                                  value=GWASummaryStatisticsDF['position'].values)
    GWASummaryStatisticsDF.insert(loc=5,
                                  column='Chr',
                                  value=GWASummaryStatisticsDF['chr'].values)
    GWASummaryStatisticsDF.insert(
        loc=14,
        column='pP-value',
        value=-pd.np.real(
            pd.np.log10(
                GWASummaryStatisticsDF['P-value'])))
    GWASummaryStatisticsDF.drop(labels=['SNPID',
                                        'chr',
                                        'position'],
                                axis=1,
                                inplace=True)

    AxesDict = {}

    AxesDict['Chr'] = {}
    AxesDict['Chr']['Label'] = 'Chromosome'
    Chromosomes = VariantInfoDF['chr'].unique()
    Chromosomes.sort()
    Offset = 10000
    AxesDict['Chr']['Min'] = float(VariantInfoDF.loc[
            VariantInfoDF['chr']==Chromosomes[0],
            'position'].min() - Offset)
    AxesDict['Chr']['MajorTickRotation'] = 0
    AxesDict['Chr']['MajorTickLabels'] = []
    AxesDict['Chr']['MajorTickLabels'].append('')
    AxesDict['Chr']['MajorTickPositions'] = []
    AxesDict['Chr']['MajorTickPositions'].append(AxesDict['Chr']['Min'])
    AxesDict['Chr']['MinorTickRotation'] = 90
    AxesDict['Chr']['MinorTickLabels'] = []
    AxesDict['Chr']['MinorTickPositions'] = []
    for i in xrange(1,
                    len(Chromosomes)):
        Chr = Chromosomes[i]
        AxesDict['Chr']['MajorTickLabels'].append('')
        AxesDict['Chr']['MajorTickPositions'].append(
            AxesDict['Chr']['MajorTickPositions'][-1] + 
            2.0*Offset +
            float(VariantInfoDF.loc[
                      VariantInfoDF['chr']==Chromosomes[i-1],
                      'position'].max() -
                  VariantInfoDF.loc[
                      VariantInfoDF['chr']==Chromosomes[i-1],
                      'position'].min()))
        AxesDict['Chr']['MinorTickLabels'].append(str(Chromosomes[i-1]))
        Range = AxesDict['Chr']['MajorTickPositions'][-1] - \
            AxesDict['Chr']['MajorTickPositions'][-2]
        AxesDict['Chr']['MinorTickPositions'].append(
            AxesDict['Chr']['MajorTickPositions'][-2] + 
            0.5*Range)  
    AxesDict['Chr']['MajorTickLabels'].append('')
    AxesDict['Chr']['MajorTickPositions'].append(
        AxesDict['Chr']['MajorTickPositions'][-1] + 
        2.0*Offset +
        float(VariantInfoDF.loc[
                  VariantInfoDF['chr']==Chromosomes[-1],
                  'position'].max() -
              VariantInfoDF.loc[
                  VariantInfoDF['chr']==Chromosomes[-1],
                  'position'].min()))
    AxesDict['Chr']['MinorTickLabels'].append(str(Chromosomes[-1]))
    Range = AxesDict['Chr']['MajorTickPositions'][-1] - \
        AxesDict['Chr']['MajorTickPositions'][-2]
    AxesDict['Chr']['MinorTickPositions'].append(
        AxesDict['Chr']['MajorTickPositions'][-2] + 
        0.5*Range)
    AxesDict['Chr']['Max'] = AxesDict['Chr']['MajorTickPositions'][-1]

    AxesDict['Trait'] = {}
    AxesDict['Trait']['Label'] = 'Trait'
    AxesDict['Trait']['Min'] = float(TraitAttributesDF['Index'].min() - 1)
    AxesDict['Trait']['Max'] = float(TraitAttributesDF['Index'].max() + 1)
    AxesDict['Trait']['MajorTickRotation'] = 0
    AxesDict['Trait']['MajorTickLabels'] = TraitAttributesDF['ENGAGEName'].tolist()
    AxesDict['Trait']['MajorTickPositions'] = TraitAttributesDF['Index'].astype(float).tolist()
    AxesDict['Trait']['MinorTickRotation'] = 0
    AxesDict['Trait']['MinorTickLabels'] = [''] * (len(AxesDict['Trait']['MajorTickLabels'])+1)
    AxesDict['Trait']['MinorTickPositions'] = TraitAttributesDF['Index'].astype(float) - 0.5
    AxesDict['Trait']['MinorTickPositions'] = AxesDict['Trait']['MinorTickPositions'].tolist()
    AxesDict['Trait']['MinorTickPositions'].append(AxesDict['Trait']['MinorTickPositions'][-1]+1.0)

    AxesDict['pP'] = {}
    AxesDict['pP']['Label'] = r'$\mathsf{-\log_{10}\left[p-value\right]}$'
    AxesDict['pP']['Min'] = 0.0
    AxesDict['pP']['Max'] = GWASummaryStatisticsDF['pP-value'].max() + 5.0
    AxesDict['pP']['MajorTickRotation'] = 0
    AxesDict['pP']['MajorTickLabels'] = pd.np.arange(start=AxesDict['pP']['Min'],
                                                         stop=(AxesDict['pP']['Max']+5.0),
                                                         step=10.0).astype(str).tolist()
    AxesDict['pP']['MajorTickPositions'] = pd.np.arange(start=AxesDict['pP']['Min'],
                                                         stop=(AxesDict['pP']['Max']+5.0),
                                                         step=10.0).tolist()
    AxesDict['pP']['MinorTickRotation'] = 0
    AxesDict['pP']['MinorTickLabels'] = [''] * (len(AxesDict['pP']['MajorTickLabels'])-1)
    AxesDict['pP']['MinorTickPositions'] = pd.np.arange(start=AxesDict['pP']['Min'],
                                                            stop=(AxesDict['pP']['Max']+5.0),
                                                            step=10.0) + 5.0
    AxesDict['pP']['MinorTickPositions'] = AxesDict['pP']['MinorTickPositions'].tolist()[:-1]
    
    AxesDF = pd.DataFrame(data=AxesDict)
    
    FName = 'AxesDF.pckl'
    AxesDF.to_pickle(os.path.join(TmpPath,
                                  FName))
    AxesDF.to_pickle(os.path.join(GWADataPath,
                                  FName))
    del FName
    
    GWASummaryStatisticsDF.insert(loc=7,
                                  column='MHPos',
                                  value=GWASummaryStatisticsDF['Pos'].astype(float).values)
    Chromosomes = GWASummaryStatisticsDF['Chr'].unique()
    Chromosomes.sort()
    for Chr in Chromosomes:
        Filter = GWASummaryStatisticsDF['Chr'] == Chr
        GWASummaryStatisticsDF.loc[Filter,
                                   'MHPos'] = \
            GWASummaryStatisticsDF.loc[Filter,
                                       'MHPos'] + \
                AxesDF.loc['MajorTickPositions',
                           'Chr'][Chr-1] + \
                Offset
        del Filter
        
    GWASummaryStatisticsDF = GWASummaryStatisticsDF.merge(right=TraitAttributesDF[['Trait',
                                                                                   'Class']],
                                                          left_on='Trait',
                                                          right_on='Trait',
                                                          how='left')
    
    GWASummaryStatisticsDF['TraitIndex'] = 0
    for T in GWASummaryStatisticsDF['Trait'].unique():
        Filter = GWASummaryStatisticsDF['Trait'] == T
        GWASummaryStatisticsDF.loc[Filter,
                                   'TraitIndex'] = TraitAttributesDF.loc[TraitAttributesDF['ENGAGEName']==T,
                                                                         'Index'].values[0]
        del Filter
    
    FName = 'GWASummaryStatisticsDF.pckl'
    GWASummaryStatisticsDF.to_pickle(os.path.join(TmpPath,
                                                  FName))
    GWASummaryStatisticsDF.to_pickle(os.path.join(GWADataPath,
                                                  FName))
    del FName

In [ ]:
GWASummaryStatisticsDF=GWASummaryStatisticsDF.sort_values(by=['Chr'])
GWASummaryStatisticsDF

,Index,Trait,MarkerName,Allele1,Allele2,Chr,Pos,MHPos,Freq1,FreqSE,...,StdErr,P-value,pP-value,Direction,HetISq,HetChiSq,HetDf,HetPVal,Class,TraitIndex
422,94,C10.1,rs6687882,A,C,1,76137413,7.618258e+07,0.3836,0.0353,...,0.0063,2.184000e-22,21.660747,+++++,22.1,5.133,4.0,0.27390,acylcarnitines,3
470,140,C10.1,rs11164042,A,G,1,75951028,7.599619e+07,0.2856,0.0251,...,0.0069,1.381000e-11,10.859806,+++++,37.7,6.418,4.0,0.17000,acylcarnitines,3
471,141,C10.1,rs12140121,A,G,1,76227589,7.627275e+07,0.7563,0.0147,...,0.0071,3.674000e-25,24.434861,+++++,13.7,4.633,4.0,0.32700,acylcarnitines,3
472,142,C10.1,rs11588643,A,G,1,76028201,7.607336e+07,0.7233,0.0274,...,0.0069,1.089000e-11,10.962972,-----,59.4,9.840,4.0,0.04321,acylcarnitines,3
473,143,C10.1,rs1498315,A,G,1,75931813,7.597698e+07,0.3153,0.0169,...,0.0066,4.010000e-33,32.396856,-----,41.1,6.791,4.0,0.14740,acylcarnitines,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710,15,Pro,rs2518805,A,G,22,17355587,2.769982e+09,0.8968,0.0103,...,0.0082,2.997000e-48,47.523313,-------,10.7,6.721,6.0,0.34740,aminoacids,26
1711,16,Pro,rs2518803,C,G,22,17355446,2.769982e+09,0.1006,0.0109,...,0.0083,2.073000e-49,48.683401,+++++++,0.0,5.608,6.0,0.46850,aminoacids,26
1713,18,Pro,rs2518802,A,C,22,17355345,2.769982e+09,0.8990,0.0105,...,0.0083,1.569000e-50,49.804377,-------,0.0,4.362,6.0,0.62780,aminoacids,26
1715,20,Pro,rs2238754,T,C,22,17438146,2.770065e+09,0.0831,0.0068,...,0.0113,4.884000e-08,7.311224,--+----,52.3,12.585,6.0,0.05012,aminoacids,26


In [ ]:
type(GWASummaryStatisticsDF)

pandas.core.frame.DataFrame

In [ ]:
GWASummaryStatisticsDF['Chr'].min()

1

In [ ]:
GWASummaryStatisticsDF['Chr']

422      1
470      1
471      1
472      1
473      1
        ..
1710    22
1711    22
1713    22
1715    22
1704    22
Name: Chr, Length: 5463, dtype: int64

In [ ]:
GWASummaryStatisticsDF.columns

Index(['Index', 'Trait', 'MarkerName', 'Allele1', 'Allele2', 'Chr', 'Pos',
       'MHPos', 'Freq1', 'FreqSE', 'MinFreq', 'MaxFreq', 'Effect', 'StdErr',
       'P-value', 'pP-value', 'Direction', 'HetISq', 'HetChiSq', 'HetDf',
       'HetPVal', 'Class', 'TraitIndex'],
      dtype='object')

In [ ]:
GWASummaryStatisticsDF[['MHPos','P-value','pP-value','TraitIndex']]

,MHPos,P-value,pP-value,TraitIndex
422,7.618258e+07,2.184000e-22,21.660747,3
470,7.599619e+07,1.381000e-11,10.859806,3
471,7.627275e+07,3.674000e-25,24.434861,3
472,7.607336e+07,1.089000e-11,10.962972,3
473,7.597698e+07,4.010000e-33,32.396856,3
...,...,...,...,...
1710,2.769982e+09,2.997000e-48,47.523313,26
1711,2.769982e+09,2.073000e-49,48.683401,26
1713,2.769982e+09,1.569000e-50,49.804377,26
1715,2.770065e+09,4.884000e-08,7.311224,26


### pP-value Threshold of 100

In [ ]:
significant_Pval_DF = GWASummaryStatisticsDF['pP-value'] >= 100

In [ ]:
significant_Pval = GWASummaryStatisticsDF[significant_Pval_DF]

In [ ]:
significant_Pval

,Index,Trait,MarkerName,Allele1,Allele2,Chr,Pos,MHPos,Freq1,FreqSE,...,StdErr,P-value,pP-value,Direction,HetISq,HetChiSq,HetDf,HetPVal,Class,TraitIndex
1289,179,C9,rs7601356,T,C,2,210764902,4.579806e+08,0.6351,0.0047,...,0.0090,5.910000e-118,117.228413,----,61.8,7.855,3.0,0.049100,acylcarnitines,18
1185,75,C9,rs3764913,T,C,2,210783154,4.579988e+08,0.6305,0.0079,...,0.0089,5.470000e-105,104.262013,----,77.2,13.178,3.0,0.004266,acylcarnitines,18
1252,142,C9,rs7557847,A,G,2,210841395,4.580571e+08,0.3648,0.0058,...,0.0091,7.690000e-110,109.114074,++++,44.4,5.399,3.0,0.144800,acylcarnitines,18
1240,130,C9,rs2286963,T,G,2,210768295,4.579840e+08,0.6338,0.0067,...,0.0089,2.820000e-118,117.549751,----,62.6,8.012,3.0,0.045760,acylcarnitines,18
1238,128,C9,rs3738934,T,C,2,210582491,4.577982e+08,0.6227,0.0072,...,0.0090,6.960000e-105,104.157391,----,24.1,3.952,3.0,0.266700,acylcarnitines,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,3,PC.aa.C38.4,rs174536,A,C,11,61308503,1.873991e+09,0.6777,0.0215,...,0.0051,2.620000e-168,167.581699,+++++++,53.3,12.836,6.0,0.045720,glycerophospholipids,54
2661,5,PC.aa.C38.4,rs4246215,T,G,11,61320875,1.874004e+09,0.3416,0.0233,...,0.0052,1.020000e-147,146.991400,-------,72.5,21.785,6.0,0.001324,glycerophospholipids,54
2662,6,PC.aa.C38.4,rs174535,T,C,11,61307932,1.873991e+09,0.6774,0.0218,...,0.0051,7.850000e-168,167.105130,+++++++,52.4,12.596,6.0,0.049910,glycerophospholipids,54
2666,10,PC.aa.C38.4,rs174574,A,C,11,61356918,1.874040e+09,0.3237,0.0224,...,0.0051,8.180000e-169,168.087247,-------,55.8,13.581,6.0,0.034680,glycerophospholipids,54


In [ ]:
type(AxesDF)

pandas.core.frame.DataFrame

In [ ]:
colors = ['#E24E42','#008F95']

In [ ]:
print(colors)

['#E24E42', '#008F95']


In [ ]:
AxesDF.columns

Index(['Chr', 'Trait', 'pP'], dtype='object')

In [ ]:
AxesDF[['Chr', 'Trait', 'pP']]

,Chr,Trait,pP
Label,Chromosome,Trait,$\mathsf{-\log_{10}\left[p-value\right]}$
MajorTickLabels,"[, , , , , , , , , , , , , , , , , , , , , , ]","[C0, C10, C10.1, C10.2, C12.1, C14.1, C14.1.OH...","[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0..."
MajorTickPositions,"[35162.0, 247205690.0, 489966477.0, 689352029....","[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","[0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0..."
MajorTickRotation,0,0,0
Max,2.78779e+09,130,179.69
Min,35162,0,0
MinorTickLabels,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[, , , , , , , , , , , , , , , , , , , , , , ,...","[, , , , , , , , , , , , , , , , , ]"
MinorTickPositions,"[123620426.0, 368586083.5, 589659253.0, 784991...","[0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, ...","[5.0, 15.0, 25.0, 35.0, 45.0, 55.0, 65.0, 75.0..."
MinorTickRotation,90,0,0


### Define Chromosome tick properties `ChrTicks` (`list` of `tuple`s):

In [ ]:
ChrTicks = []
for i in range(len(AxesDF.loc['MinorTickPositions',
                               'Chr'])):
    ChrTicks.append((AxesDF.loc['MinorTickPositions',
                                'Chr'][i],
                     AxesDF.loc['MinorTickLabels',
                                'Chr'][i]))  
ChrTicks

[(123620426.0, '1'),
 (368586083.5, '2'),
 (589659253.0, '3'),
 (784991889.0, '4'),
 (971021605.0, '5'),
 (1146818303.5, '6'),
 (1311579672.0, '7'),
 (1464028431.0, '8'),
 (1607232475.0, '9'),
 (1745007493.0, '10'),
 (1879824905.5, '11'),
 (2013122131.5, '12'),
 (2127380089.5, '13'),
 (2219579656.0, '14'),
 (2304704661.5, '15'),
 (2390150190.0, '16'),
 (2473886198.5, '17'),
 (2551282369.5, '18'),
 (2621234015.0, '19'),
 (2684338975.0, '20'),
 (2734087993.5, '21'),
 (2770200701.5, '22')]

In [ ]:
GWASummaryStatisticsDF_copy =GWASummaryStatisticsDF.copy()

In [ ]:
(GWASummaryStatisticsDF_copy)

,Index,Trait,MarkerName,Allele1,Allele2,Chr,Pos,MHPos,Freq1,FreqSE,...,StdErr,P-value,pP-value,Direction,HetISq,HetChiSq,HetDf,HetPVal,Class,TraitIndex
422,94,C10.1,rs6687882,A,C,1,76137413,7.618258e+07,0.3836,0.0353,...,0.0063,2.184000e-22,21.660747,+++++,22.1,5.133,4.0,0.27390,acylcarnitines,3
470,140,C10.1,rs11164042,A,G,1,75951028,7.599619e+07,0.2856,0.0251,...,0.0069,1.381000e-11,10.859806,+++++,37.7,6.418,4.0,0.17000,acylcarnitines,3
471,141,C10.1,rs12140121,A,G,1,76227589,7.627275e+07,0.7563,0.0147,...,0.0071,3.674000e-25,24.434861,+++++,13.7,4.633,4.0,0.32700,acylcarnitines,3
472,142,C10.1,rs11588643,A,G,1,76028201,7.607336e+07,0.7233,0.0274,...,0.0069,1.089000e-11,10.962972,-----,59.4,9.840,4.0,0.04321,acylcarnitines,3
473,143,C10.1,rs1498315,A,G,1,75931813,7.597698e+07,0.3153,0.0169,...,0.0066,4.010000e-33,32.396856,-----,41.1,6.791,4.0,0.14740,acylcarnitines,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710,15,Pro,rs2518805,A,G,22,17355587,2.769982e+09,0.8968,0.0103,...,0.0082,2.997000e-48,47.523313,-------,10.7,6.721,6.0,0.34740,aminoacids,26
1711,16,Pro,rs2518803,C,G,22,17355446,2.769982e+09,0.1006,0.0109,...,0.0083,2.073000e-49,48.683401,+++++++,0.0,5.608,6.0,0.46850,aminoacids,26
1713,18,Pro,rs2518802,A,C,22,17355345,2.769982e+09,0.8990,0.0105,...,0.0083,1.569000e-50,49.804377,-------,0.0,4.362,6.0,0.62780,aminoacids,26
1715,20,Pro,rs2238754,T,C,22,17438146,2.770065e+09,0.0831,0.0068,...,0.0113,4.884000e-08,7.311224,--+----,52.3,12.585,6.0,0.05012,aminoacids,26


In [ ]:
GWASummaryStatisticsDF_copy['p_adj'] = -np.log10(GWASummaryStatisticsDF_copy['P-value'])

In [ ]:
GWASummaryStatisticsDF_copy['Chr'] = (GWASummaryStatisticsDF_copy['Chr'].astype('category')).cat.as_ordered()

In [ ]:
#GWASummaryStatisticsDF_copy.Trait.unique()

In [ ]:
if "APOA1" in GWASummaryStatisticsDF_copy.values:
    print('Element exists in Dataframe')
else:
    print("gene Not available")

gene Not available


In [ ]:
GWASummaryStatisticsDF_copy['ind'] = range(len(GWASummaryStatisticsDF_copy))

In [ ]:
GWASummaryStatisticsDF_copy

,Index,Trait,MarkerName,Allele1,Allele2,Chr,Pos,MHPos,Freq1,FreqSE,...,pP-value,Direction,HetISq,HetChiSq,HetDf,HetPVal,Class,TraitIndex,p_adj,ind
422,94,C10.1,rs6687882,A,C,1,76137413,7.618258e+07,0.3836,0.0353,...,21.660747,+++++,22.1,5.133,4.0,0.27390,acylcarnitines,3,21.660747,0
470,140,C10.1,rs11164042,A,G,1,75951028,7.599619e+07,0.2856,0.0251,...,10.859806,+++++,37.7,6.418,4.0,0.17000,acylcarnitines,3,10.859806,1
471,141,C10.1,rs12140121,A,G,1,76227589,7.627275e+07,0.7563,0.0147,...,24.434861,+++++,13.7,4.633,4.0,0.32700,acylcarnitines,3,24.434861,2
472,142,C10.1,rs11588643,A,G,1,76028201,7.607336e+07,0.7233,0.0274,...,10.962972,-----,59.4,9.840,4.0,0.04321,acylcarnitines,3,10.962972,3
473,143,C10.1,rs1498315,A,G,1,75931813,7.597698e+07,0.3153,0.0169,...,32.396856,-----,41.1,6.791,4.0,0.14740,acylcarnitines,3,32.396856,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710,15,Pro,rs2518805,A,G,22,17355587,2.769982e+09,0.8968,0.0103,...,47.523313,-------,10.7,6.721,6.0,0.34740,aminoacids,26,47.523313,5458
1711,16,Pro,rs2518803,C,G,22,17355446,2.769982e+09,0.1006,0.0109,...,48.683401,+++++++,0.0,5.608,6.0,0.46850,aminoacids,26,48.683401,5459
1713,18,Pro,rs2518802,A,C,22,17355345,2.769982e+09,0.8990,0.0105,...,49.804377,-------,0.0,4.362,6.0,0.62780,aminoacids,26,49.804377,5460
1715,20,Pro,rs2238754,T,C,22,17438146,2.770065e+09,0.0831,0.0068,...,7.311224,--+----,52.3,12.585,6.0,0.05012,aminoacids,26,7.311224,5461


In [ ]:
GWASummaryStatisticsDF_copy['p_adj'].min(), GWASummaryStatisticsDF_copy['p_adj'].max()

(7.30111686324741, 174.6903698325741)

In [ ]:
GWASummaryStatisticsDF_copy[['Index','Trait','Chr','p_adj','ind']]

,Index,Trait,Chr,p_adj,ind
422,94,C10.1,1,21.660747,0
470,140,C10.1,1,10.859806,1
471,141,C10.1,1,24.434861,2
472,142,C10.1,1,10.962972,3
473,143,C10.1,1,32.396856,4
...,...,...,...,...,...
1710,15,Pro,22,47.523313,5458
1711,16,Pro,22,48.683401,5459
1713,18,Pro,22,49.804377,5460
1715,20,Pro,22,7.311224,5461


In [ ]:
df_grouped = GWASummaryStatisticsDF_copy.groupby('Chr')

In [ ]:
df_grouped.size()

Chr
1      646
2      560
3        8
4       59
5      536
6      255
7       35
8        1
10      90
11    2583
12      26
13       6
14     289
15      13
16     126
17      16
18      47
19      49
20      86
22      32
dtype: int64

In [ ]:
df_grouped.count()

,Index,Trait,MarkerName,Allele1,Allele2,Pos,MHPos,Freq1,FreqSE,MinFreq,...,pP-value,Direction,HetISq,HetChiSq,HetDf,HetPVal,Class,TraitIndex,p_adj,ind
Chr,,,,,,,,,,,,,,,,,,,,,
1,646,646,646,646,646,646,646,646,646,646,...,646,646,646,646,646,646,646,646,646,646
2,560,560,560,560,560,560,560,560,560,560,...,560,560,560,560,560,560,560,560,560,560
3,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
4,59,59,59,59,59,59,59,59,59,59,...,59,59,59,59,59,59,59,59,59,59
5,536,536,536,536,536,536,536,536,536,536,...,536,536,536,536,536,536,536,536,536,536
6,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
7,35,35,35,35,35,35,35,35,35,35,...,35,35,35,35,35,35,35,35,35,35
8,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
10,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,90


In [ ]:
type(df_grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [ ]:
x_labels = []
x_labels_pos = []
chrList =[]
grouped_dataFrame = pd.DataFrame()

for num, (name, group) in enumerate(df_grouped):
    #px.scatter(group, x='ind', y='p_adj', color="Chr")
    grouped_dataFrame = grouped_dataFrame.append(group)
    x_labels.append(name)
    x_labels_pos.append((group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0])/2))
    

In [ ]:
# Pickling the DF
pickle_groupDF = GWASummaryStatisticsDF_copy

with open('groupedDF.pickl','wb') as DF_pickl:
    pickle.dump(pickle_groupDF, DF_pickl, pickle.HIGHEST_PROTOCOL)
    

In [ ]:
x_labels

[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22]

In [ ]:
grouped_dataFrame['Chr']

422      1
470      1
471      1
472      1
473      1
        ..
1710    22
1711    22
1713    22
1715    22
1704    22
Name: Chr, Length: 5463, dtype: category
Categories (20, int64): [1 < 2 < 3 < 4 ... 18 < 19 < 20 < 22]

In [ ]:
min(x_labels), max(x_labels)

(1, 22)

In [ ]:
grouped_dataFrame['Chr'].min(), grouped_dataFrame['Chr'].max()

(1, 22)

In [ ]:
grouped_dataFrame["Index"].max(), grouped_dataFrame["Index"].min()

(351, 0)

In [ ]:
grouped_dataFrame.columns

Index(['Index', 'Trait', 'MarkerName', 'Allele1', 'Allele2', 'Chr', 'Pos',
       'MHPos', 'Freq1', 'FreqSE', 'MinFreq', 'MaxFreq', 'Effect', 'StdErr',
       'P-value', 'pP-value', 'Direction', 'HetISq', 'HetChiSq', 'HetDf',
       'HetPVal', 'Class', 'TraitIndex', 'p_adj', 'ind'],
      dtype='object')

In [ ]:
#df_grouped.get_group(11)
significance = (0.05/5463)
significance1 = np.(1*e-8)
significance1

SyntaxError: invalid syntax (<ipython-input-104-8316468f3814>, line 3)

In [ ]:
fig= px.scatter()
refSNP = None
significance = -np.log10(0.05/5463)

fig= px.scatter(data_frame=grouped_dataFrame, 
                       x='ind', 
                       y='p_adj', 
                       color='Chr', 
                       symbol= None, 
                       size= None, 
                       hover_name='Pos', 
                       hover_data= None, #'Pos',
                       custom_data=None, 
                       text=None, 
                       facet_row=None, 
                       #facet_col='Chr', 
                       facet_col_wrap=0, 
                       error_x=None, 
                       error_x_minus=None, 
                       error_y=None, 
                       error_y_minus=None, 
                       animation_frame=None, 
                       animation_group=None, 
                       category_orders={}, 
                       labels={'ind':'Data Point', 
                               'p_adj':'-log(P-Value)',
                               'pP-value':'-log(P-Value)',
                               'Chr':'Chr',
                               'Pos':'Position'}, 
                       color_discrete_sequence=None, 
                       color_discrete_map={}, 
                       color_continuous_scale=None, 
                       range_color=None, 
                       color_continuous_midpoint=None, 
                       symbol_sequence=None, 
                       symbol_map={}, 
                       opacity=None, 
                       size_max=5, 
                       marginal_x=None, 
                       marginal_y=None, 
                       trendline=None, 
                       trendline_color_override=None, 
                       log_x=False, 
                       log_y=False, 
                       range_x= [min(x_labels_pos), max(x_labels_pos)], 
                       range_y= None, 
                       render_mode='auto', 
                       title="Manhattan Plot", 
                       template=None, 
                       width=None, 
                       height= 600)

fig.update_xaxes(rangemode="tozero", 
                 showgrid=False, 
                 #zeroline=True
                showline=True, 
                 linewidth=2, 
                 linecolor='black', 
                 mirror=True,
                 nticks= max(x_labels),
                 tickvals= x_labels_pos,
                 ticks="outside", 
                 tickwidth=2, 
                 tickcolor='#9E9E9E', 
                 ticklen=10,
                 ticktext = x_labels
                #range=[1, grouped_dataFrame['Chr'].max()]
                )
fig.update_yaxes(rangemode="tozero", 
                 showgrid= True,#False, 
                 #zeroline=True,
                showline=True, 
                 linewidth=2, 
                 linecolor='black', 
                 mirror=True,
                                
                )
fig.update_layout( xaxis = dict( range=(min(x_labels_pos)-360, 
                                        max(x_labels_pos)+111),
                                        constrain='domain'),
                 
                  #xaxis_type='category'
                 )

fig.add_shape( 
    # add a horizontal "target" line
    type="line", 
    line_color="blue", 
    line_width=1, 
    opacity=1, 
    #line_dash="dot",
    x0=0, 
    x1=1, 
    xref="paper",
    y0=160, 
    y1=160, 
    yref="y"
)

# fig.add_shape(
#         # Line Vertical
#         dict(
#             type="line",
#             x0=1,
#             y0=0,
#             x1=1,
#             y1=2,
#             line=dict(
#                 color="RoyalBlue",
#                 width=3
#             )

# if refSNP:
#         for index, row in data.iterrows():
#             if row['-log10(p_value)'] >= significance:
#                 ax.annotate(str(row[refSNP]), xy = (index, row['-log10(p_value)']))

In [ ]:
#fig.show()

### KEGG Database ###

In [ ]:
#Start a kegg interface (default organism is human, that is called hsa):
import networkx as nx

In [ ]:
k = KEGG() # Initilaise the KEGG DB

### Creating an genes Objects from stored file in the hardrive 

In [ ]:
fileOfGenes= with open('genelist.txt', 'r+')
genelist = [line.strip("\n") for line in fileOfGenes.readlines()]
fileOfGenes.close()
genelist

['FADS1', 'CPS1', 'ACADL', 'FADS2', 'FADS3']

### Pickling the gene list

In [ ]:
genelist_pickl_object = genelist

#exampleObj = {'Python':3,'KDE':5,'Windows':10} # Creating a python object

with open('genedata.pickle', 'wb') as genelist_file_Obj:
    # Creating pickle file object
    pickle.dump(genelist_pickl_object, genelist_file_Obj, pickle.HIGHEST_PROTOCOL) # Dumping the object into the pickle file

genelist_file_Obj.close() #close the file

### Deserialize the pickle object

In [ ]:
with open('genedata.pickle', 'rb') as genelist_file_Obj:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    
    genelist_pickl_object = pickle.load(genelist_file_Obj)

genelist_file_Obj.close()

In [ ]:
print(genelist_pickl_object)

['FADS1', 'CPS1', 'ACADL', 'FADS2', 'FADS3']


### Chekcing if the pckl file exist on the hard-drive

In [ ]:
#File = os.path('genedata.pickle')
import time
if os.path.isfile('genedata.pickle'):
    
    print ("___ The KEGG DB Pathway Object Files existing ____")
    
    print("___ Skipping Pathway Extraction from KEGG DB _____ ")
    
    Date_modified = time.ctime(os.path.getmtime("pathways.obj"))
    
    print("Last modified: %s" % Date_modified)
    
    Date_created = time.ctime(os.path.getctime("pathways.obj"))
    
    print("Created: %s" % Date_created )

else:
    print ("File not exist")

#import os.path, time



___ The KEGG DB Pathway Object Files existing ____
___ Skipping Pathway Extraction from KEGG DB _____ 
Last modified: Sun May 17 05:55:31 2020
Created: Tue May  5 13:02:14 2020


### Looking  for pathway (by genes i.e., IDs or usual name)

In [ ]:
def keggpathwayDB():

    export_all_list = True

    path_network = nx.DiGraph()

    k = KEGG()

    run_gene_list = genelist_pickl_object

    #run_gene_list = ['FADS1', 'FADS2', 'FADS3']

    k.organism = "hsa"

    out_file_name = 'combined_network'

    list_pathways = []

    list_res = []

    list_entry = []
    
    path_relation = []

    for a_gene in run_gene_list:  

        #print(a_gene)

        #k.find("hsa", a_gene)

        pathways = k.get_pathway_by_gene(a_gene, "hsa")

        #print(pathways)

        if pathways != None:

            list_pathways.append(pathways)

            for a_pathway in pathways.keys():

                # search for pathways that contain the required gene Id and relations
                res = k.parse_kgml_pathway(a_pathway) 

                list_res.append(a_pathway)
                
                path_relation.append(res)

                #print(res.keys())

                gene_id = None

                for entry in res['entries']:             

                    if entry['gene_names'] != None:

                        if a_gene in entry['gene_names'].split(', '):

                            list_entry.append(entry)

                            #print(entry)

                            gene_id = entry['id']
                           
print ("____Done Execution!____")

return(list_pathways, list_res, list_entry)

____Done Execution!____


##### Serialise the pathways

In [ ]:
pickle_pathway_Obj = list_pathways

pickle_dict_res_Obj = list_res

pickle_list_entries = list_entry

file_pathway_Obj = with open('pathways.obj', 'wb') # Creating pickle file object

file_dict_res_Obj = with open('res.obj','wb')

file_list_entry_Obj = with open('entries.obj','wb')

pickle.dump(pickle_pathway_Obj,file_pathway_Obj) # Dumping the object into the pickle file

pickle.dump(pickle_dict_res_Obj,file_dict_res_Obj)

pickle.dump(pickle_list_entries,file_list_entry_Obj)

file_pathway_Obj.close() #close the file

file_dict_res_Obj.close()

file_list_entry_Obj.close()

#### Deserialize object

In [ ]:
#def deseralisePcklFile():
with open('pathways.obj', 'rb') as ds_file_pathway_Obj:

    pickle_pathway_Obj = pickle.load(ds_file_pathway_Obj)

    ds_file_pathway_Obj.close()

with open('res.obj','rb') as ds_file_dict_res_Obj:

    pickle_dict_res_Obj = pickle.load(ds_file_dict_res_Obj)

    ds_file_dict_res_Obj.close()


with open("entries.obj",'rb') as ds_file_list_entry_Obj:

    pickle_list_entry_Obj = pickle.load(ds_file_list_entry_Obj)

    ds_file_list_entry_Obj.close()

#return (pickle_pathway_Obj, pickle_dict_res_Obj, pickle_list_entry_Obj)

In [ ]:
#deseralisePcklFile()
pathways_list = pd.DataFrame(pickle_list_entry_Obj)

pathways_list_Duplicate_removed = pathways_list.drop_duplicates(['name'], 
                                                                keep= 'first', 
                                                                inplace= False)

pathways_list_Duplicate_removed = pathways_list_Duplicate_removed.reset_index(drop =True)

pathways_list_Duplicate_removed = pathways_list_Duplicate_removed.rename(columns =
                                                                         {'name':'entry_name(s)'})
pathways_list_Duplicate_removed

,id,entry_name(s),type,link,gene_names
0,308,hsa:3992,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:3992,"FADS1, D5D, FADS6, FADSD5, LLCDL1, TU12"
1,73,hsa:9415,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:9415,"FADS2, D6D, DES6, FADSD6, LLCDL2, SLL0262, TU13"
2,48,hsa:1373,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:1373,"CPS1, CPSASE1, PHN"
3,2368,hsa:1589,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:1589,"CYP21A2, CA21H, CAH1, CPS1, CYP21, CYP21B, P45..."
4,61,hsa:33,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33,"ACADL, ACAD4, LCAD"
5,2234,hsa:33 hsa:34 hsa:37 hsa:51 hsa:51102 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD..."
6,2242,hsa:33 hsa:34 hsa:51 hsa:51102 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD..."
7,2257,hsa:33 hsa:34 hsa:35 hsa:36 hsa:51 hsa:51102 h...,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD..."
8,312,hsa:33 hsa:34 hsa:35 hsa:36 hsa:51 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD..."
9,313,hsa:33 hsa:34 hsa:51 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD..."


In [ ]:
#for (k, v) in pathways_list.items():
  #  print (k,v)
unique_path_ID=[]
for pathwayid in pickle_dict_res_Obj:
    if pathwayid not in unique_path_ID:
        unique_path_ID.append(pathwayid)
        
DFunique_pathways_id = pd.DataFrame(unique_path_ID, columns =['pathway_Name'])
#DFunique_pathways_id

### Add the pathway ID to the DF

In [ ]:
pathways_list_Duplicate_removed['pathway_name'] = unique_path_ID
pathways_list_Duplicate_removed

,id,entry_name(s),type,link,gene_names,path_human_name,pathway_name
0,308,hsa:3992,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:3992,"FADS1, D5D, FADS6, FADSD5, LLCDL1, TU12",Biosynthesis of unsaturated fatty acids,hsa01040
1,73,hsa:9415,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:9415,"FADS2, D6D, DES6, FADSD6, LLCDL2, SLL0262, TU13",Metabolic pathways,hsa01100
2,48,hsa:1373,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:1373,"CPS1, CPSASE1, PHN",Fatty acid metabolism,hsa01212
3,2368,hsa:1589,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:1589,"CYP21A2, CA21H, CAH1, CPS1, CYP21, CYP21B, P45...",alpha-Linolenic acid metabolism,hsa00592
4,61,hsa:33,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33,"ACADL, ACAD4, LCAD",PPAR signaling pathway,hsa03320
5,2234,hsa:33 hsa:34 hsa:37 hsa:51 hsa:51102 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD...",Arginine biosynthesis,hsa00220
6,2242,hsa:33 hsa:34 hsa:51 hsa:51102 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD...","Alanine, aspartate and glutamate metabolism",hsa00250
7,2257,hsa:33 hsa:34 hsa:35 hsa:36 hsa:51 hsa:51102 h...,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD...",Nitrogen metabolism,hsa00910
8,312,hsa:33 hsa:34 hsa:35 hsa:36 hsa:51 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD...",Carbon metabolism,hsa01200
9,313,hsa:33 hsa:34 hsa:51 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD...",Biosynthesis of amino acids,hsa01230


In [ ]:
pathwaysgenes = pd.DataFrame(pickle_pathway_Obj)
pathwaysgenes

,hsa01040,hsa01100,hsa01212,hsa00592,hsa03320,hsa00220,hsa00250,hsa00910,hsa01200,hsa01230,hsa00071
0,Biosynthesis of unsaturated fatty acids,Metabolic pathways,Fatty acid metabolism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Biosynthesis of unsaturated fatty acids,Metabolic pathways,Fatty acid metabolism,alpha-Linolenic acid metabolism,PPAR signaling pathway,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Metabolic pathways,NaN,NaN,NaN,Arginine biosynthesis,"Alanine, aspartate and glutamate metabolism",Nitrogen metabolism,Carbon metabolism,Biosynthesis of amino acids,NaN
3,NaN,Metabolic pathways,Fatty acid metabolism,NaN,PPAR signaling pathway,NaN,NaN,NaN,NaN,NaN,Fatty acid degradation


### Adding human readable to the DF

In [ ]:
path_values =[]
path_keys = []
for dicts in range(len(pickle_pathway_Obj)):
    if dicts != None:
        for key, value in (pickle_pathway_Obj[dicts]).items():
            #if key, value not in path_name_ID:
            path_values.append(value)
            path_keys.append(key)
# print(path_values) 
# print(path_keys)

pathDF = pd.DataFrame(path_values, columns =['path_human_name'])
pathDF['pathway_name'] = path_keys

#pathDF

path_values = list(dict.fromkeys(path_values ) ) ## Removing duplicates from the list

# same asbelow
pathDF_Duplicate_removed = pathDF.drop_duplicates(['path_human_name'], 
                                                                keep= 'first', 
                                                                inplace= False)

pathDF_Duplicate_removed = pathDF_Duplicate_removed.reset_index(drop =True)
#print(path_values)
#print(pathDF_Duplicate_removed)

pathways_list_Duplicate_removed['path_human_name'] = path_values
pathways_list_Duplicate_removed

,id,entry_name(s),type,link,gene_names,path_human_name,pathway_name
0,308,hsa:3992,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:3992,"FADS1, D5D, FADS6, FADSD5, LLCDL1, TU12",Biosynthesis of unsaturated fatty acids,hsa01040
1,73,hsa:9415,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:9415,"FADS2, D6D, DES6, FADSD6, LLCDL2, SLL0262, TU13",Metabolic pathways,hsa01100
2,48,hsa:1373,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:1373,"CPS1, CPSASE1, PHN",Fatty acid metabolism,hsa01212
3,2368,hsa:1589,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:1589,"CYP21A2, CA21H, CAH1, CPS1, CYP21, CYP21B, P45...",alpha-Linolenic acid metabolism,hsa00592
4,61,hsa:33,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33,"ACADL, ACAD4, LCAD",PPAR signaling pathway,hsa03320
5,2234,hsa:33 hsa:34 hsa:37 hsa:51 hsa:51102 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD...",Arginine biosynthesis,hsa00220
6,2242,hsa:33 hsa:34 hsa:51 hsa:51102 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD...","Alanine, aspartate and glutamate metabolism",hsa00250
7,2257,hsa:33 hsa:34 hsa:35 hsa:36 hsa:51 hsa:51102 h...,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD...",Nitrogen metabolism,hsa00910
8,312,hsa:33 hsa:34 hsa:35 hsa:36 hsa:51 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD...",Carbon metabolism,hsa01200
9,313,hsa:33 hsa:34 hsa:51 hsa:8310,gene,http://www.kegg.jp/dbget-bin/www_bget?hsa:33+h...,"ACADL, ACAD4, LCAD...",Biosynthesis of amino acids,hsa01230


In [ ]:
print(pathDF_Duplicate_removed)

                                path_human_name pathway_name
0       Biosynthesis of unsaturated fatty acids     hsa01040
1                            Metabolic pathways     hsa01100
2                         Fatty acid metabolism     hsa01212
3               alpha-Linolenic acid metabolism     hsa00592
4                        PPAR signaling pathway     hsa03320
5                         Arginine biosynthesis     hsa00220
6   Alanine, aspartate and glutamate metabolism     hsa00250
7                           Nitrogen metabolism     hsa00910
8                             Carbon metabolism     hsa01200
9                   Biosynthesis of amino acids     hsa01230
10                       Fatty acid degradation     hsa00071


In [ ]:
'''
def GenerateManhattan(pyhattan_object, export_path = None, significance = 6, colors = ['#E24E42', '#008F95'], refSNP = False):
    data = pyhattan_object[0]
    data_grouped = pyhattan_object[1]

    fig = plt.figure()
    ax = fig.add_subplot(111)

    x_labels = []
    x_labels_pos = []
    for num, (name, group) in enumerate(data_grouped):
        #group.plot(kind='scatter', x='ind', y='-log10(p_value)', color=colors[num % len(colors)], ax=ax, s= 10000/len(data))
        group.plot(kind='scatter', x='ind', y='-log10(p_value)', color=colors[num % len(colors)], ax=ax, s= 10000/len(data))
        x_labels.append(name)
        x_labels_pos.append((group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0]) / 2))

    ax.set_xticks(x_labels_pos)
    ax.set_xticklabels(x_labels)
    ax.set_xlim([0, len(data)])
    ax.set_ylim([0, data['-log10(p_value)'].max() + 1])
    ax.set_xlabel('Chromosome')
    plt.axhline(y=significance, color='gray', linestyle='-', linewidth = 0.5)
    plt.xticks(fontsize=8, rotation=60)
    plt.yticks(fontsize=8)

    if refSNP:
        for index, row in data.iterrows():
            if row['-log10(p_value)'] >= significance:
                ax.annotate(str(row[refSNP]), xy = (index, row['-log10(p_value)']))

    if export_path:
        plt.savefig(export_path)

    plt.show()
'''

## Processing the File with gene names #

In [ ]:
Genefile = 'LeadSNPPerLocus.tsv'
GeneFileDF = pd.read_csv(os.path.join(GWADataPath,
                                                     Genefile),
                                        index_col=None,
                                        sep='\t')

In [ ]:
GeneFileDF

,LocusName,LeadMetabolomicSNP,LeadMetabolite
0,FADS1-3,rs174547,lysoPC a C20:4
1,THEM4,rs10494270,C9
2,TMEM229B,rs1077989,PC ae C32:1
3,SLC16A9,rs1171614,C0
4,SLC22A16_SLC16A10,rs12210538,C18:2
5,PAH,rs12297049,Phe
6,GCKR,rs1260326,PC aa C40:5
7,CERS4,rs12610250,SM C18:0
8,PDXDC1_PLA2G10,rs6498540,lysoPC a C20:3
9,CPS1_ACADL,rs2286963,C9


### Getting significant genes

In [ ]:
signSNPS =[]
for snps in GeneFileDF["LeadMetabolomicSNP"]:
    for snpss in significant_Pval['MarkerName']:
        if snps == snpss:
            if snps not in signSNPS:
                signSNPS.append(snps)
        
print("The list of unique SNPS are: ", signSNPS)       
 

The list of unique SNPS are:  ['rs174547', 'rs2286963']


In [ ]:
GeneFileDF_genes_significant = GeneFileDF.loc[GeneFileDF['LeadMetabolomicSNP'].isin(signSNPS)]

In [ ]:
GeneFileDF_genes_significant.index =[0,1]
GeneFileDF_genes_significant

,LocusName,LeadMetabolomicSNP,LeadMetabolite
0,FADS1-3,rs174547,lysoPC a C20:4
1,CPS1_ACADL,rs2286963,C9


In [ ]:
# Explode/Split Gene column name (LocusName) into multiple rows

new_GeneFile_Series = GeneFileDF_genes_significant['LocusName'].str.split('_').apply(pd.Series, 1).stack()

#new_GeneFile_Series = GeneFileDF_genes_significant['LocusName'].str.split('-').apply(pd.Series, 1).stack()
# Get rid of the stack:
# Drop the level to line up with the DataFrame

new_GeneFile_Series.index= new_GeneFile_Series.index.droplevel(-1)
#new_GeneFile_Series

# Make your series` a dataframe 
new_GeneFileDF = pd.DataFrame(new_GeneFile_Series)
new_GeneFileDF.columns =['gene']
new_GeneFileDF1= new_GeneFileDF['gene'].str.split('-').apply(pd.Series, 1).stack()
new_GeneFileDF = pd.DataFrame(new_GeneFileDF1, columns =['gene'])
new_GeneFileDF2= new_GeneFileDF.reset_index(drop=True)
new_GeneFileDF2.drop([1], inplace=True)
new_GeneFileDF3 = new_GeneFileDF2.reset_index(drop=True)
new_GeneFileDF3

,gene
0,FADS1
1,CPS1
2,ACADL


### Adding condensed gene names to the original lists if genes 

In [ ]:
# Building an added list

added_gene_list = ['FADS2','FADS3']

added_genes = pd.DataFrame(added_gene_list, columns=["gene"])

added_genes

,gene
0,FADS2
1,FADS3


### Concatnating the two DF

In [ ]:
new_GeneFileDF4 = new_GeneFileDF3.append(added_genes)
new_GeneFileDF4 = new_GeneFileDF4.reset_index(drop=True)
new_GeneFileDF4

,gene
0,FADS1
1,CPS1
2,ACADL
3,FADS2
4,FADS3


## Saving the gene names on a txt file

In [ ]:
genelists = new_GeneFileDF4['gene'].values.tolist() # Coverting the DF into a list

In [ ]:
genefile = open('genelist.txt', 'w')
 
# List of numbers
genelist = genelists
''' 
Write the genes by one to a file 

'''
for i in genelist:
    genefile.write(str(i) + "\n")
 
# Close the file
genefile.close()


In [1]:
import pandas as pd

certificates_earned = pd.Series(
    [8, 2, 5, 6],
    index=['Tom', 'Kris', 'Ahmad', 'Beau']
)

print(certificates_earned)


Tom      8
Kris     2
Ahmad    5
Beau     6
dtype: int64


In [4]:
import pandas as pd

certificates_earned = pd.Series(
    [8, 2, 5, 6],
    index=['Tom', 'Kris', 'Ahmad', 'Beau']
)

print(certificates_earned[certificates_earned > 5])

Tom     8
Beau    6
dtype: int64


In [10]:
import pandas as pd

certificates_earned = pd.DataFrame({
    'Certificates': [8, 2, 5, 6],
    'Time (in months)': [16, 5, 9, 12]
})

certificates_earned.index = ['Tom', 'Kris', 'Ahmad', 'Beau']

print(certificates_earned.iloc[2])

Certificates        5
Time (in months)    9
Name: Ahmad, dtype: int64


In [9]:
certificates_earned

,Certificates,Time (in months)
Tom,8,16
Kris,2,5
Ahmad,5,9
Beau,6,12
